In [6]:
import caffe
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

caffe.set_device(0)
caffe.set_mode_gpu()

import cv2

from math import ceil,log10
from __future__ import division  # py2
from __future__ import print_function

In [7]:
n_is_64 = True

In [9]:
# model_prototxt = 'nets/compress_v{}_exp6.prototxt'.format(1 if n_is_64 else 2) # v1 for n=64, v2 for n=128 
model_prototxt = 'nets/compress_v{}.prototxt'.format(1 if n_is_64 else 2) # v1 for n=64, v2 for n=128 

In [4]:
# model_prototxt = 'nets/compress_v1_test_caffe_crop.prototxt'

In [9]:
custom_test_imgs_filelist = "ctifl.txt"  # Kodak images

In [10]:
def revert_to_img(dec_img):
    dec_img = np.clip(dec_img.data[0]*127.5+127.5, 0, 255)
    dec_img = cv2.cvtColor(np.moveaxis(dec_img, 0, -1), cv2.COLOR_BGR2RGB)
    return dec_img.astype(np.uint8)

#### For evaluate psnr of Kodak

In [11]:
model_nums = 4 if n_is_64 else 6
start_id = 0 if n_is_64 else 4
imgs_num = len(open(custom_test_imgs_filelist).readlines())
print ('total imgs_nums = %d. ' % imgs_num)
calc_psnr = lambda x,y: 10 * log10(255.0**2/(np.mean(np.square(y-x))))


for model_id in range(start_id, model_nums):
    model_id += 1
    print ('Evaluating model %d.' % model_id)
    model_weights = 'ckpts/%d.caffemodel' % model_id
    net = caffe.Net(model_prototxt, caffe.TEST, stages=['custom'], weights=model_weights)
    batch_size = 1
    iters_num = int(ceil(imgs_num / batch_size))
    mse_loss = 0
    rate_disp = 0
    psnr = 0
    for it in range(iters_num):
        net.forward()
        mse_loss += float(net.blobs['loss'].data) / iters_num
        rate_disp += float(net.params['imap'][0].data) / iters_num

        o_img = revert_to_img(net.blobs['data'])
        d_img = revert_to_img(net.blobs['pdata'])
        psnr += calc_psnr(o_img, d_img) / iters_num

    print (' %f/%f ' % (mse_loss, rate_disp))
    print (' psnr = %f ' % (psnr,))
    print ('=' * 10)
    

total imgs_nums = 23. 
Evaluating model 5.
 1425.151553/0.339828 
 psnr = 33.940022 
Evaluating model 6.
 1350.446408/0.395653 
 psnr = 34.134717 


#### caffe mse loss & rate on KITTI val/test subset

In [10]:
model_nums = 4 if n_is_64 else 6
start_id = 0 if n_is_64 else 4

for model_id in range(start_id, model_nums):
    model_id += 1
    print ('Evaluating model %d.' % model_id)
    model_weights = 'ckpts/%d.caffemodel' % model_id
    for val_mode in [True, False]:
        net = caffe.Net(model_prototxt, caffe.TEST, stages=['val'] if val_mode else [], weights=model_weights)
        imgs_num = 240 if val_mode else 241
        batch_size = 1
        iters_num = int(ceil(imgs_num / batch_size))
        mse_loss = 0
        rate_disp = 0
        for it in range(iters_num):
            net.forward()
            mse_loss += float(net.blobs['loss'].data) / iters_num
            rate_disp += float(net.params['imap'][0].data) / iters_num
        print (' %f/%f ' % (mse_loss, rate_disp))
        print ('=' * 10)
    

Evaluating model 1.
 582.830076/0.188000 
 591.597566/0.189805 
Evaluating model 2.
 510.373779/0.257809 
 519.832220/0.260280 
Evaluating model 3.
 396.082002/0.428965 
 402.839909/0.432637 
Evaluating model 4.
 370.835725/0.663635 
 377.765991/0.658786 


In [6]:
model_weights = 'ckpts/1.caffemodel'

In [7]:
val_mode = True  # True for val, False for test

In [8]:
net = caffe.Net(model_prototxt, caffe.TEST, stages=['val'] if val_mode else [], weights=model_weights)

In [9]:
imgs_num = 240 if val_mode else 241

In [10]:
batch_size = 1

In [11]:
iters_num = int(ceil(imgs_num / batch_size))

In [12]:
mse_loss = 0
rate_disp = 0

In [14]:
# net.forward()
# 943.6048583984375 val 验证是否确实完成一轮 epoch

# 500.8824157714844 test

{'loss': array(500.8824157714844, dtype=float32)}

In [17]:
for it in range(iters_num-4):
#     print ('Iter %d' % it)
    net.forward()
    mse_loss += float(net.blobs['loss'].data) / iters_num
    rate_disp += float(net.params['imap'][0].data) / iters_num

In [13]:
print ('mse loss = %f, rate_disp = %f' % (mse_loss, rate_disp))
print (' %f/%f ' % (mse_loss, rate_disp))

mse loss = 591.597566, rate_disp = 0.189805
 591.597566/0.189805 


<h4> 测试caffe python接口的backward()是否会更新参数(<font style="color:red">不会</font>)，以及是否会累积梯度(<font style="color:red">会</font>)</h4>

In [41]:
net.forward()

{'loss': array(699.9473266601562, dtype=float32)}

In [13]:
net.backward()

{}

In [49]:
for layer_name, param in net.params.items():
#     print (param)
    print (layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape) if len(param)>1 else "")

conv1	(128, 3, 8, 8) (128,)
blk1_branch2b	(128, 128, 3, 3) (128,)
blk1_branch2c	(128, 128, 3, 3) (128,)
conv2	(256, 128, 4, 4) (256,)
blk2_branch2b	(128, 256, 3, 3) (128,)
blk2_branch2c	(256, 128, 3, 3) (256,)
conv3	(256, 256, 3, 3) (256,)
blk3_branch2b	(128, 256, 3, 3) (128,)
blk3_branch2c	(256, 128, 3, 3) (256,)
conv4	(64, 256, 1, 1) (64,)
mgdata	(1, 1, 1, 1) 
imp_conv1	(128, 256, 3, 3) (128,)
imp_conv2	(1, 128, 1, 1) (1,)
imap	(1, 1, 1, 1) 
inv_conv1	(512, 64, 3, 3) (512,)
blk4_branch2b	(128, 512, 3, 3) (128,)
blk4_branch2c	(512, 128, 3, 3) (512,)
inv_conv2	(512, 512, 3, 3) (512,)
blk5_branch2b	(128, 512, 3, 3) (128,)
blk5_branch2c	(512, 128, 3, 3) (512,)
inv_conv3	(256, 128, 3, 3) (256,)
blk6_branch2b	(128, 256, 3, 3) (128,)
blk6_branch2c	(256, 128, 3, 3) (256,)
inv_conv4	(32, 16, 3, 3) (32,)
pdata	(3, 32, 1, 1) (3,)


In [ ]:
net.params['conv1'][0].data

In [45]:
net.params['conv1'][0].diff[...]=0

In [ ]:
net.params['conv1'][0].diff